# **【分類】キノコの有毒／無毒を予測する**

#1.準備作業

※0章のコマンドはLinuxコマンドであることを留意すること

##ドライブのマウント

■ドライブをマウントする

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


□結果：「Mounted at /content/drive」と表示されること

---



##ディレクトリの作成

■ディレクトリが存在しないことを確認する

In [ ]:
ls -la /content | grep data

drwxr-xr-x 1 root root 4096 Mar  9 14:48 sample_data/


□結果：「data」ディレクトリが存在しないこと

■ディレクトリを作成する

In [ ]:
mkdir /content/data

□結果：エラーが発生しないこと

■ディレクトリが作成されたことを確認する

In [ ]:
ls -la /content | grep data

drwxr-xr-x 2 root root 4096 Mar 15 14:43 data/
drwxr-xr-x 1 root root 4096 Mar  9 14:48 sample_data/


□結果：「data」ディレクトリが存在すること

##カレントディレクトの移動

■カレントディレクトリを移動する

In [ ]:
cd /content/data

/content/data


□結果：「/content/data」と表示されること

##データの格納

■/content/dataディレクトリにmushrooms.csvを手動で格納する

□結果：エラーが発生しないこと

■格納したファイルが存在することを確認する

In [ ]:
ls -la | grep mushrooms.csv

-rw-r--r-- 1 root root 1229339 Mar 15 14:44 mushrooms.csv


□結果：「mushrooms.csv」が存在すること

#2.前処理とチューニング

###2.1.CSVファイルの読み込み

In [ ]:
#pandasをインポート
import pandas as pd

#mushroom.csvのデータを読み込む
df = pd.read_csv('mushrooms.csv')

#データの内容を確認
df

,class,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,poisonous,convex,smooth,brown,yes,pungent,free,close,narrow,black,...,smooth,white,white,partial,white,one,pendant,black,scattered,urban
1,edible,convex,smooth,yellow,yes,almond,free,close,broad,black,...,smooth,white,white,partial,white,one,pendant,brown,numerous,grasses
2,edible,bell,smooth,white,yes,anise,free,close,broad,brown,...,smooth,white,white,partial,white,one,pendant,brown,numerous,meadows
3,poisonous,convex,scaly,white,yes,pungent,free,close,narrow,brown,...,smooth,white,white,partial,white,one,pendant,black,scattered,urban
4,edible,convex,smooth,gray,no,none,free,crowded,broad,black,...,smooth,white,white,partial,white,one,evanescent,brown,abundant,grasses
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,edible,knobbed,smooth,brown,no,none,attached,close,broad,yellow,...,smooth,orange,orange,partial,orange,one,pendant,buff,clustered,leaves
8120,edible,convex,smooth,brown,no,none,attached,close,broad,yellow,...,smooth,orange,orange,partial,brown,one,pendant,buff,several,leaves
8121,edible,flat,smooth,brown,no,none,attached,close,broad,brown,...,smooth,orange,orange,partial,orange,one,pendant,buff,clustered,leaves
8122,poisonous,knobbed,scaly,brown,no,fishy,free,close,narrow,buff,...,silky,white,white,partial,white,one,evanescent,white,several,leaves


##2.2.欠損値処理

In [ ]:
#欠損値の数を確認する
df.isnull().sum()

class                       0
cap-shape                   0
cap-surface                 0
cap-color                   0
bruises                     0
odor                        0
gill-attachment             0
gill-spacing                0
gill-size                   0
gill-color                  0
stalk-shape                 0
stalk-root                  0
stalk-surface-above-ring    0
stalk-surface-below-ring    0
stalk-color-above-ring      0
stalk-color-below-ring      0
veil-type                   0
veil-color                  0
ring-number                 0
ring-type                   0
spore-print-color           0
population                  0
habitat                     0
dtype: int64

※欠損値が存在しない為、欠損値処理は不要

In [ ]:
#(欠損値は存在しないが)元データと区別する為に欠損値なし変数にコピー
non_df = df.copy()

##2.3.外れ値処理

In [ ]:
#ユニークな要素と要素の数を出力
for col in non_df.columns:
    print(non_df[col].value_counts())

edible       4208
poisonous    3916
Name: class, dtype: int64
convex     3656
flat       3152
knobbed     828
bell        452
sunken       32
conical       4
Name: cap-shape, dtype: int64
scaly           3244
smooth          2556
fibrous         2320
groovesmooth       4
Name: cap-surface, dtype: int64
brown       2284
gray        1840
red         1500
yellow      1072
white       1040
buff         168
pink         144
cinnamon      44
purple        16
green         16
Name: cap-color, dtype: int64
no     4748
yes    3376
Name: bruises, dtype: int64
none        3528
foul        2160
fishy        576
spicy        576
almond       400
anise        400
pungent      256
creosote     192
musty         36
Name: odor, dtype: int64
free        7914
attached     210
Name: gill-attachment, dtype: int64
close      6812
crowded    1312
Name: gill-spacing, dtype: int64
broad     5612
narrow    2512
Name: gill-size, dtype: int64
buff         1728
pink         1492
white        1202
brown        1048

※外れ値は存在しない為、外れ値処理は不要  
※「stalk-color-above-ring」の「e」と、「stalk-color-below-ring」の「e」は、「red」への変換ミスだと思われる。

In [ ]:
#(外れ値は存在しないが)元データと区別する為に外れ値なし変数にコピー
non_noo_df = non_df.copy()

##2.4.ダミー変数化

【説明】  
■名義尺度  
他と区別し分類するための名称のようなもの  
例：男女、血液型、郵便番号、住所、本籍地、所属学部、学籍番号  
使える統計量：各ケースの数、計数（count）、頻度（frequency）、最頻値、連関係数  
対処法：get_dummiesを用いてダミー変数化する。その際、変数同士の相関が高いと多重共線性という問題が発生するため、回避するためのオプション（drop_first=True）を使用する。

In [ ]:
#名義尺度の列をダミー変数化
non_noo_df = pd.get_dummies(non_noo_df, drop_first=True, columns=["cap-shape","cap-surface","cap-color","bruises","odor","gill-attachment","gill-color","stalk-shape","stalk-root","stalk-surface-above-ring","stalk-surface-below-ring","stalk-color-above-ring","stalk-color-below-ring","veil-type","veil-color","ring-type","spore-print-color","habitat"])

#データの内容を確認
non_noo_df

,class,gill-spacing,gill-size,ring-number,population,cap-shape_conical,cap-shape_convex,cap-shape_flat,cap-shape_knobbed,cap-shape_sunken,...,spore-print-color_orange,spore-print-color_purple,spore-print-color_white,spore-print-color_yellow,habitat_leaves,habitat_meadows,habitat_paths,habitat_urban,habitat_waste,habitat_woods
0,poisonous,close,narrow,one,scattered,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,edible,close,broad,one,numerous,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,edible,close,broad,one,numerous,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,poisonous,close,narrow,one,scattered,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,edible,crowded,broad,one,abundant,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,edible,close,broad,one,clustered,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
8120,edible,close,broad,one,several,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
8121,edible,close,broad,one,clustered,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
8122,poisonous,close,narrow,one,several,0,0,0,1,0,...,0,0,1,0,1,0,0,0,0,0


□結果：名義尺度として指定した列がダミー変数化されていること

##2.5.カテゴリ変数化

【説明】  
■順序尺度  
順序や大小には意味があるが間隔には意味がないもの  
例えば、1位＋2位≠3位のように、足し算引き算ができないもの  
例：1位 / 2位 / 3位…、1. 好き / 2. ふつう / 3. 嫌い、統計検定®1級 / 2級 / 3級 / 4級、がんのステージ分類におけるステージI / II / III / IV  
使える統計量：中央値、パーセンタイル  
対処法：LabelEncoderを用いてカテゴリ変数化する。

In [ ]:
#sklearnをインポート
from sklearn import preprocessing

#順序尺度の列をカテゴリ変数化
for column in ["class", "gill-spacing", "gill-size", "ring-number", "population"]:
    selected_column = non_noo_df[column]
    #LabelEncoderでカテゴリ変数化する
    le = preprocessing.LabelEncoder()
    le.fit(selected_column)
    column_le = le.transform(selected_column)
    #カテゴリ変数化した値でnon_noo_df内のデータを書き換える
    non_noo_df[column] = pd.Series(column_le).astype('category')

#データの内容を確認
non_noo_df

,class,gill-spacing,gill-size,ring-number,population,cap-shape_conical,cap-shape_convex,cap-shape_flat,cap-shape_knobbed,cap-shape_sunken,...,spore-print-color_orange,spore-print-color_purple,spore-print-color_white,spore-print-color_yellow,habitat_leaves,habitat_meadows,habitat_paths,habitat_urban,habitat_waste,habitat_woods
0,1,0,1,1,3,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,0,0,0,1,2,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,2,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,1,0,1,1,3,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,0,1,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8119,0,0,0,1,1,0,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
8120,0,0,0,1,4,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0
8121,0,0,0,1,1,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
8122,1,0,1,1,4,0,0,0,1,0,...,0,0,1,0,1,0,0,0,0,0


□結果：順序尺度として指定した列がカテゴリ変数化されていること

##2.6.データの分割

In [ ]:
#train_test_splitをインポート
from sklearn.model_selection import train_test_split

#正解データ：1列目(class列(毒の有無))
t = non_noo_df.iloc[:, 0]

#特徴量：2列目以降(class列以外)
x = non_noo_df.iloc[:, 1:]

#学習データとテストデータに分割
(x_train, x_test, y_train, y_test) = train_test_split(x, t, test_size = 0.2, random_state = 0)

#分割されたデータ数の表示
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(6499, 90) (1625, 90) (6499,) (1625,)


#3.学習

##3.1.決定木モデルでの実装

###3.1.0.learn関数の実装

In [ ]:
#treeのインポート
from sklearn import tree

# learn関数
def learn(x, t, depth = 3):
    #テストデータと訓練データへ分割
    x_train, x_test, y_train, y_test = train_test_split(x, t, test_size = 0.2, random_state = 0)
    #学習
    model = tree.DecisionTreeClassifier(max_depth = depth, random_state = 0, class_weight = "balanced")
    model.fit(x_train, y_train)

    score = model.score(X = x_train, y = y_train)
    score2 = model.score(X = x_test, y = y_test)
    return round(score, 3), round(score2, 3), model

###3.1.1.学習

In [ ]:
# learn関数を用いて深さごとに学習し、正解率と精度を確認
for j in range(1, 16):
    # learn関数による学習
    train_score, test_score, model = learn(x, t, depth = j)

    #正解率と精度の出力
    total_sentence = '深さ{:2} \t 訓練データの正解率{:.3f} \t テストデータの精度{:.3f}'
    print(total_sentence.format(j, train_score, test_score))

深さ 1 	 訓練データの正解率0.889 	 テストデータの精度0.879
深さ 2 	 訓練データの正解率0.954 	 テストデータの精度0.956
深さ 3 	 訓練データの正解率0.986 	 テストデータの精度0.983
深さ 4 	 訓練データの正解率0.994 	 テストデータの精度0.990
深さ 5 	 訓練データの正解率0.997 	 テストデータの精度0.994
深さ 6 	 訓練データの正解率0.997 	 テストデータの精度0.994
深さ 7 	 訓練データの正解率1.000 	 テストデータの精度1.000
深さ 8 	 訓練データの正解率1.000 	 テストデータの精度1.000
深さ 9 	 訓練データの正解率1.000 	 テストデータの精度1.000
深さ10 	 訓練データの正解率1.000 	 テストデータの精度1.000
深さ11 	 訓練データの正解率1.000 	 テストデータの精度1.000
深さ12 	 訓練データの正解率1.000 	 テストデータの精度1.000
深さ13 	 訓練データの正解率1.000 	 テストデータの精度1.000
深さ14 	 訓練データの正解率1.000 	 テストデータの精度1.000
深さ15 	 訓練データの正解率1.000 	 テストデータの精度1.000


In [ ]:
#深さ7以降で正解率と正答率が最大となっているため、深さ7を採用する
# 学習
model = tree.DecisionTreeClassifier(max_depth = 7, random_state = 0)
model.fit(x_train, y_train)

DecisionTreeClassifier(max_depth=7, random_state=0)

###3.1.2.予測

In [ ]:
#テストデータを渡して予測結果を取得
predict = model.predict(x_test)

#予測結果の出力
predict

array([1, 0, 0, ..., 1, 0, 0])

###3.1.3.精度

In [ ]:
#metricsをインポート
from sklearn import metrics 

#予測結果の精度を確認
ac_score = metrics.accuracy_score(y_test, predict)
cl_report = metrics.classification_report(y_test, predict)
print("正解率 =", ac_score)
print("レポート =\n", cl_report)

正解率 = 1.0
レポート =
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       852
           1       1.00      1.00      1.00       773

    accuracy                           1.00      1625
   macro avg       1.00      1.00      1.00      1625
weighted avg       1.00      1.00      1.00      1625



##3.2.ランダムフォレストでの実装

###3.2.1.学習

In [ ]:
#RandomForestClassifierのインポート
from sklearn.ensemble import RandomForestClassifier

# 学習
model = RandomForestClassifier()
model.fit(x_train, y_train)

RandomForestClassifier()

###3.2.2.予測

In [ ]:
#テストデータを渡して予測結果を取得
predict = model.predict(x_test)

#予測結果の出力
predict

array([1, 0, 0, ..., 1, 0, 0])

###3.2.3.精度

In [ ]:
#metricsをインポート
from sklearn import metrics 

#予測結果の精度を確認
ac_score = metrics.accuracy_score(y_test, predict)
cl_report = metrics.classification_report(y_test, predict)
print("正解率 =", ac_score)
print("レポート =\n", cl_report)

正解率 = 1.0
レポート =
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       852
           1       1.00      1.00      1.00       773

    accuracy                           1.00      1625
   macro avg       1.00      1.00      1.00      1625
weighted avg       1.00      1.00      1.00      1625



#4.モデルの保存

In [ ]:
#pickleをインポート
import pickle

#モデルを保存
with open('mushrooms.pkl', 'wb') as f:
    pickle.dump(model, f)

In [ ]:
ls -l | grep mushrooms.pkl

-rw-r--r-- 1 root root  555931 Mar 15 14:51 mushrooms.pkl


□結果：「mushrooms.pkl」が存在すること

#5.備考

※欠損値が存在しない為、欠損値処理は不要  
※正解データはclass列を用いる  
※特徴量はclass列を除く全ての列を使用できる為、特徴量の絞り込みは不要  
※十分な正解率と精度を有する為、多項式特徴量や交互作用特徴量の追加は不要